# DeepCAD-RT testing pipeline 
<img src="https://github.com/STAR-811/DeepCAD-RT-old/blob/main/images/logo-new.png?raw=true" width = "650" height = "180" align=right />
This file will demonstrate pipeline for testing microscopy data using the DeepCAD-RT algorithm.<br>
The demo shows how to construct the params and call the relevant functions for testing DeepCAD-RT network. In addition, it will automatically download tif file and corresponding model file for demo testing.<br>

More information can be found in the companion paper.
See inside for details.

In [1]:
from deepcad.test_collection import testing_class
from deepcad.movie_display import display
from deepcad.utils import get_first_filename, download_demo

### Select file(s) to be processed (download if not present)
The `download_demo` function will download the specific file for you and return the complete path to the file which will be stored in your `datasets` directory. If you adapt this demo for your data make sure to pass the datasets folder name of your file(s).

In [2]:
download_demo_file = True
if download_demo_file:
    file_name='simulate_-2.51dBSNR_1000frames' # select the demo file you want to test (e.g. 'ATP_3D', 'fish_localbrain', 'NP_3D', ...)
    datasets_path, denoise_model =download_demo(download_filename=file_name)
else:
    datasets_path = 'datasets/2RPN-1000'  # folder containing files for testing
    denoise_model = '2RPN_202112101128_ov_0.5' # A folder containing models to be tested

Downloading...
From: https://zenodo.org/record/5790790/files/noise_2RPN_-2.51dBSNR_1000frames_.tif?download=1
To: E:\01-LYX\pipPackage\DeepCAD_RT_pytorch\notebooks\datasets\simulate_-2.51dBSNR_1000frames_demo\simulate_-2.51dBSNR_1000frames.tif
100%|████████████████████████████████████████████████████████████████████████████████| 486M/486M [08:07<00:00, 998kB/s]
Downloading...
From: https://zenodo.org/record/5790790/files/E_13_Iter_6198.pth?download=1
To: E:\01-LYX\pipPackage\DeepCAD_RT_pytorch\notebooks\pth\simulate_-2.51dBSNR_1000frames_best_model_demo\best_model.pth
100%|█████████████████████████████████████████████████████████████████████████████| 4.10M/4.10M [00:02<00:00, 1.56MB/s]
Downloading...
From: https://zenodo.org/record/5790790/files/para.yaml?download=1
To: E:\01-LYX\pipPackage\DeepCAD_RT_pytorch\notebooks\pth\simulate_-2.51dBSNR_1000frames_best_model_demo\best_model.yaml
100%|██████████████████████████████████████████████████████████████████████████████████| 292/292 [00:0

### Set up some parameters for testing
Default setting shows the parameters suitable for demo file. You can change the testing parameters accroding to your testing data and device. For supervising the testing process, you can set the flag for visualization and result storage to `True`.

In [4]:
test_datasize = 300                   # the number of slices to be tested
GPU = '0'                             # the index of GPU you will use for computation (e.g. '0', '0,1', '0,1,2')
patch_xy = 150              # the width and height of 3D patches
patch_t = 150               # the time dimension of 3D patches
overlap_factor = 0.4                  # the overlap factor between two adjacent patches
num_workers = 0                       # if you use Windows system, set this to 0.

# Setup some parameters for result visualization during testing period (optional)
visualize_images_per_epoch = False  # choose whether to display inference performance after each epoch
save_test_images_per_epoch = True  # choose whether to save inference image after each epoch in pth path

###  Play the demo noisy movie (optional)
Play the first noisy movie (optional). This will require loading the movie in memory which in general is not needed by the pipeline. Displaying the movie uses the OpenCV library. 

In [5]:
display_images = True

if display_images:
    display_filename = get_first_filename(datasets_path)
    print('\033[1;31mDisplaying the first raw file -----> \033[0m')
    print(display_filename)
    display_length = 300  # the frames number of the noise movie
    # normalize the image and display
    display(display_filename, display_length=display_length, norm_min_percent=1, norm_max_percent=98)

Displaying the first raw file -----> 
datasets/simulate_-2.51dBSNR_1000frames_demo/simulate_-2.51dBSNR_1000frames.tif


### Create a testing class object with the specified parameters
You will creat a parameters object by passing all the parameters as a single dictionary. Parameters not defined in the dictionary will assume their default values.

In [6]:
test_dict = {
    # dataset dependent parameters
    'patch_x': patch_xy,                # you can change these params if use anisotropy patch size
    'patch_y': patch_xy,
    'patch_t': patch_t,
    'overlap_factor':overlap_factor,
    'scale_factor': 1,                   # the factor for image intensity scaling
    'test_datasize': test_datasize,
    'datasets_path': datasets_path,
    'pth_dir': './pth',                 # pth file root path
    'denoise_model' : denoise_model,
    'output_dir' : './results',         # result file root path
    # network related parameters
    'fmap': 16,                          # number of feature maps
    'GPU': GPU,
    'num_workers': num_workers,
    'visualize_images_per_epoch': visualize_images_per_epoch,
    'save_test_images_per_epoch': save_test_images_per_epoch
}

tc = testing_class(test_dict)

Testing parameters -----> 
{'overlap_factor': 0.5, 'datasets_path': 'datasets/simulate_-2.51dBSNR_1000frames_demo', 'fmap': 16, 'output_dir': './results', 'pth_dir': './pth', 'batch_size': 1, 'patch_t': 150, 'patch_x': 150, 'patch_y': 150, 'gap_y': 75, 'gap_x': 75, 'gap_t': 75, 'GPU': '0', 'ngpu': 1, 'num_workers': 0, 'scale_factor': 1, 'test_datasize': 300, 'denoise_model': 'simulate_-2.51dBSNR_1000frames_best_model_demo', 'visualize_images_per_epoch': False, 'save_test_images_per_epoch': True}


### Start the testing process

In [7]:
tc.run()

Stacks for processing -----> 
Total stack number ----->  1
simulate_-2.51dBSNR_1000frames.tif
Using 1 GPU(s) for testing -----> 
[Model 1/1, best_model.pth] [Stack 1/1, simulate_-2.51dBSNR_1000frames.tif] [Patch 108/108] [Time Cost: 15 s] [ETA: 0 s]      
 Test finished. Save all results to disk.
